In [2]:
# -*- coding: utf-8 -*-
import timm
import torch
import zipfile, os
import csv
from PIL import Image
from pathlib import Path
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Check CUDA availability
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)         # Should match the installed CUDA version

local_zip = os.path.join(os.getcwd(), 'GCD.zip')
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(os.getcwd())
zip_ref.close()

train_dir = os.path.join(os.getcwd(), 'GCD', 'train')
test_dir = os.path.join(os.getcwd(), 'GCD', 'test')

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_dir)

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Data loaders
train_dataset = CustomDataset(data_dir=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function for training and evaluation
def train_and_evaluate_model(model_name, model, train_loader, test_loader, num_epochs=10):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Create a directory to save results and models
    results_dir = "results"
    os.makedirs(results_dir, exist_ok=True)

    log_file_path = os.path.join(results_dir, f"{model_name}_log.txt")
    model_save_path = os.path.join(results_dir, f"{model_name}.pth")
    csv_file_path = os.path.join(results_dir, f"{model_name}_results.csv")

    # Initialize CSV file
    with open(csv_file_path, "w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(["Epoch", "Train Loss", "Train Accuracy", "Train Precision", "Train Recall", "Train F1-Score", "Test Accuracy", "Test Precision", "Test Recall", "Test F1-Score"])

    with open(log_file_path, "w") as log_file:
        log_file.write(f"Training model: {model_name}\n")

        # Training loop
        for epoch in range(num_epochs):
            model.train()  # Set the model to training mode
            epoch_loss = 0  # To accumulate loss for the epoch
            correct_predictions = 0
            total_samples = 0
            all_targets = []
            all_preds = []

            progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")  # Progress bar

            for inputs, targets in progress_bar:
                inputs, targets = inputs.to(device), targets.to(device)  # Move to device

                optimizer.zero_grad()  # Zero the gradients
                outputs = model(inputs)  # Forward pass
                loss = criterion(outputs, targets)  # Compute loss
                loss.backward()  # Backward pass
                optimizer.step()  # Update weights

                # Update progress bar description with the current loss
                epoch_loss += loss.item()
                progress_bar.set_postfix(loss=loss.item())

                # Calculate accuracy
                _, preds = torch.max(outputs, dim=1)
                correct_predictions += (preds == targets).sum().item()
                total_samples += targets.size(0)
                all_targets.extend(targets.tolist())
                all_preds.extend(preds.tolist())

            avg_loss = epoch_loss / len(train_loader)
            accuracy = correct_predictions / total_samples * 100

            # Calculate precision, recall, and F1-score for the train set
            precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted')

            # Log training metrics
            train_metrics = (f"Epoch [{epoch+1}/{num_epochs}], "
                             f"Train Loss: {avg_loss:.4f}, "
                             f"Train Accuracy: {accuracy:.2f}%, "
                             f"Train Precision: {precision:.2f}, "
                             f"Train Recall: {recall:.2f}, "
                             f"Train F1-Score: {f1:.2f}\n")
            print(train_metrics)
            log_file.write(train_metrics)

            # Evaluate the model on the test set after each epoch
            model.eval()  # Set model to evaluation mode
            correct_predictions = 0
            total_samples = 0
            all_targets = []
            all_preds = []

            with torch.no_grad():
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, dim=1)
                    correct_predictions += (preds == targets).sum().item()
                    total_samples += targets.size(0)
                    all_targets.extend(targets.tolist())
                    all_preds.extend(preds.tolist())

            test_accuracy = correct_predictions / total_samples * 100

            # Calculate precision, recall, and F1-score for the test set
            precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted')

            # Log test metrics
            test_metrics = (f"Test Accuracy for {model_name} after epoch {epoch+1}: {test_accuracy:.2f}%\n"
                            f"Test Precision: {precision:.2f}, Test Recall: {recall:.2f}, Test F1-Score: {f1:.2f}\n\n")
            print(test_metrics)
            log_file.write(test_metrics)

            # Write metrics to CSV
            with open(csv_file_path, "a", newline="") as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow([epoch+1, avg_loss, accuracy, precision, recall, f1, test_accuracy, precision, recall, f1])

        # Save the trained model
        torch.save(model.state_dict(), model_save_path)

# Models to train
models = [
    ('maxvit_tiny_tf_224.in1k', timm.create_model('maxvit_tiny_tf_224.in1k', pretrained=True, num_classes=7)),
    ('maxvit_tiny_rw_224.sw_in1k', timm.create_model('maxvit_tiny_rw_224.sw_in1k', pretrained=True, num_classes=7)),
    ('maxvit_small_tf_224.in1k', timm.create_model('maxvit_small_tf_224.in1k', pretrained=True, num_classes=7))
]

# Train and evaluate each model
for model_name, model in models:
    train_and_evaluate_model(model_name, model, train_loader, test_loader, num_epochs=10)


True
11.8


Epoch [1/10]: 100%|██████████| 1250/1250 [07:21<00:00,  2.83it/s, loss=0.75]  


Epoch [1/10], Train Loss: 0.8791, Train Accuracy: 66.75%, Train Precision: 0.65, Train Recall: 0.67, Train F1-Score: 0.66

Test Accuracy for maxvit_tiny_tf_224.in1k after epoch 1: 64.07%
Test Precision: 0.68, Test Recall: 0.64, Test F1-Score: 0.61




Epoch [2/10]: 100%|██████████| 1250/1250 [06:45<00:00,  3.08it/s, loss=0.372] 


Epoch [2/10], Train Loss: 0.6035, Train Accuracy: 77.09%, Train Precision: 0.76, Train Recall: 0.77, Train F1-Score: 0.77

Test Accuracy for maxvit_tiny_tf_224.in1k after epoch 2: 64.63%
Test Precision: 0.67, Test Recall: 0.65, Test F1-Score: 0.64




...

Epoch [4/10], Train Loss: 0.4736, Train Accuracy: 81.69%, Train Precision: 0.81, Train Recall: 0.82, Train F1-Score: 0.81

Test Accuracy for maxvit_tiny_tf_224.in1k after epoch 4: 69.82%
Test Precision: 0.75, Test Recall: 0.70, Test F1-Score: 0.68




Epoch [5/10]: 100%|██████████| 1250/1250 [06:45<00:00,  3.08it/s, loss=0.301] 


Epoch [5/10], Train Loss: 0.4258, Train Accuracy: 83.43%, Train Precision: 0.83, Train Recall: 0.83, Train F1-Score: 0.83

Test Accuracy for maxvit_tiny_tf_224.in1k after epoch 5: 72.19%
Test Precision: 0.74, Test Recall: 0.72, Test F1-Score: 0.71




Epoch [6/10]:  84%|████████▍ | 1048/1250 [05:39<01:05,  3.09it/s, loss=0.295] 

: 